
# 03. CatBoost 모델링 및 앙상블 (CatBoost & Soft Voting)

본 노트북은 **CatBoost Classifier**를 활용하여 CTR 예측 모델을 구축하는 과정을 다룹니다.
범주형 변수 처리에 강점이 있는 CatBoost를 기반으로, 시드(Seed) 앙상블을 통해 모델의 일반화 성능을 극대화합니다.

### 🛠️ 모델링 파이프라인
1. **Down-Sampling**: 클래스 불균형 해소를 위한 1:1 비율 샘플링 적용
2. **Feature Engineering**:
    - **Sequence Features**: 사용자 행동 이력(`seq`)을 활용한 길이, 빈도, 패턴 변수 생성
    - **Interaction Features**: 주요 범주형 변수 간의 조합(Interaction) 파생 변수
    - **Group Statistics**: ID별 상세 통계량 집계
3. **Model Training**:
    - `TossMetric` (LogLoss와 AP의 가중 결합)을 기반으로 한 학습 최적화
    - 3가지 Random Seed(42, 106, 1031)를 사용한 독립적 모델 학습
4. **Ensemble**: 학습된 모델 간의 Soft Voting을 통한 최종 확률값 산출


In [1]:

import pandas as pd
import numpy as np
import os
import random
import catboost as cb
from sklearn.model_selection import train_test_split
from sklearn.metrics import average_precision_score, log_loss, accuracy_score
from tqdm import tqdm
from itertools import combinations
import gc
import warnings

warnings.filterwarnings('ignore')

# CatBoost 설치 확인 (필요시)
try:
    import catboost
except ImportError:
    !pip install catboost


In [2]:
class CFG:
    # 데이터 경로
    DATA_PATH = '/home/konkukstat/python3/workspace/data/'
    
    # 💥 학습할 시드 리스트 (앙상블 효과)
    SEEDS = [42, 106, 1031]
    
    VALID_RATIO = 0.2
    
    # 생성할 파일명 Prefix
    OUTPUT_PREFIX = '03_CatBoost'

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

print(f"Data Path: {CFG.DATA_PATH}")

Data Path: /home/konkukstat/python3/workspace/data/


## 필수 라이브러리 임포트

이 섹션에서는 CatBoost 모델을 학습하고 평가하기 위한 필수 라이브러리들을 불러옵니다.

- **pandas, numpy**: 데이터 조작 및 수치 계산
- **catboost**: 그래디언트 부스팅 기반 분류 모델
- **sklearn**: 데이터 분할, 메트릭 계산 등
- **tqdm**: 진행 상황 표시
- **gc, random, warnings**: 메모리 관리 및 환경 설정


In [3]:

# --- 1. 클릭 확률 맵 로드 ---
try:
    df_click_prob = pd.read_excel(os.path.join(CFG.DATA_PATH, 'high_click_numbers.xlsx'))
    click_prob_map = dict(zip(df_click_prob['number'], df_click_prob['click_prob']))
except FileNotFoundError:
    print("⚠️ 'high_click_numbers.xlsx' not found. Skipping click_prob features.")
    click_prob_map = {}

# --- 2. 부정/긍정 리스트 ---
pos_list = {370, 528, 68, 561, 144, 227, 417, 442, 186, 395}
neg_list = {154, 222, 84, 498, 434, 511, 216, 497, 309, 446}

def add_seq_features(df, name="dataset"):
    # seq_len, avg_prob, seq_neg, seq_pos 리스트 생성
    seq_len, avg_prob, seq_neg, seq_pos = [], [], [], []

    for s in tqdm(df["seq"], desc=f"Processing {name}"):
        if isinstance(s, str) and s != "":
            arr = [int(x) for x in s.split(",") if x]

            # 길이
            seq_len.append(len(arr))

            # 클릭 확률 평균
            probs = [click_prob_map.get(num) for num in arr if num in click_prob_map]
            avg_prob.append(sum(probs) / len(probs) if probs else np.nan)

            # neg/pos 카운트
            seq_neg.append(sum(1 for x in arr if x in neg_list))
            seq_pos.append(sum(1 for x in arr if x in pos_list))
        else:
            seq_len.append(0)
            avg_prob.append(np.nan)
            seq_neg.append(0)
            seq_pos.append(0)

    df["seq_len"] = seq_len
    df["avg_click_prob"] = avg_prob
    df["seq_neglogcount"] = seq_neg
    df["seq_poslogcount"] = seq_pos

    # avg_click_prob 결측치 처리
    overall_avg_prob = df["avg_click_prob"].mean()
    df["avg_click_prob"].fillna(overall_avg_prob, inplace=True)

    return df

def add_groupby_features(train_df, test_df, feature_name, agg_dict):
    print(f"--- Creating features by grouping '{feature_name}' ---")
    agg_stats = train_df.groupby(feature_name).agg(agg_dict)
    new_cols = [f"{feature_name}_{col[0]}_{col[1]}" for col in agg_stats.columns]
    agg_stats.columns = new_cols
    agg_stats.reset_index(inplace=True)

    train_df = pd.merge(train_df, agg_stats, on=feature_name, how='left')
    test_df = pd.merge(test_df, agg_stats, on=feature_name, how='left')

    for col in new_cols:
        fill_value = train_df[col].mean()
        test_df[col].fillna(fill_value, inplace=True)
    
    return train_df, test_df


def add_count_features(train_df, test_df, count_cols):
    print("--- Creating Count Encoding Features ---")
    for col in count_cols:
        # Use concat to get global counts
        all_data = pd.concat([train_df[col], test_df[col]], ignore_index=True)
        count_map = all_data.value_counts().to_dict()
        
        train_df[f"{col}_count"] = train_df[col].map(count_map).fillna(0).astype(int)
        test_df[f"{col}_count"] = test_df[col].map(count_map).fillna(0).astype(int)
    return train_df, test_df


## 환경 설정 및 하이퍼파라미터 정의

**CFG 클래스**: 모델 학습에 필요한 모든 설정값을 관리
- `DATA_PATH`: 데이터 파일 저장 위치
- `SEEDS`: 3개의 난수 시드 (42, 106, 1031) - 서로 다른 가중치로 초기화된 3개 모델 학습
- `VALID_RATIO`: 검증 데이터 비율 (20%)
- `OUTPUT_PREFIX`: 저장될 예측 파일 이름 prefix

**seed_everything()**: 모든 난수 생성기를 고정하여 재현 가능한 결과 보장
- Python, NumPy, 운영체제 수준의 난수 모두 제어


In [4]:

class TossMetric:
    def is_max_optimal(self): return True
    def evaluate(self, approxes, target, weight):
        y_pred = 1.0 / (1.0 + np.exp(-approxes[0]))
        y_true = np.array(target)
        # Check imbalance
        N_1, N_0 = np.sum(y_true), len(y_true) - np.sum(y_true)
        if N_1 == 0 or N_0 == 0: return 0.5, 0
        
        # Weighted LogLoss
        weights = np.where(y_true == 1, 0.5 / N_1, 0.5 / N_0)
        y_pred_clipped = np.clip(y_pred, 1e-15, 1 - 1e-15)
        wll = log_loss(y_true, y_pred_clipped, sample_weight=weights)
        
        # AP
        ap = average_precision_score(y_true, y_pred)
        
        # Score Formula
        score = 0.5 * ap + 0.5 * (1 / (1 + wll))
        return score, 1.0
        
    def get_final_error(self, scores, weight): return np.mean(scores)


## 피처 엔지니어링: 파생 변수 생성 함수들

### 1. **high_click_numbers.xlsx 로드**
- 02_EDA.ipynb에서 생성된 파일
- 각 아이템번호별 클릭 확률 저장 (예: 아이템 370은 85% 클릭률)
- 사용자가 본 아이템들의 평균 클릭 확률을 피처로 추출하는데 사용

### 2. **pos_list, neg_list**
- 긍정/부정 아이템: 강사/조정팀이 사전에 레이블링한 아이템
- 사용자 시퀀스에서 긍정/부정 아이템 개수 카운팅

### 3. **add_seq_features()**
사용자 방문 시퀀스(`seq`)로부터 4개 파생 변수 생성:
- `seq_len`: 시퀀스 길이 (사용자가 본 광고 수)
- `avg_click_prob`: 시퀀스 내 아이템들의 평균 클릭 확률
- `seq_neglogcount`: 방문한 부정 아이템 개수
- `seq_poslogcount`: 방문한 긍정 아이템 개수

### 4. **add_groupby_features()**
특정 컬럼(예: inventory_id)으로 그룹화하여 집계 통계 생성:
- 각 인벤토리별 평균/표준편차 계산
- 학습 데이터 통계로 테스트 데이터 채우기 (데이터 누출 방지)

### 5. **add_count_features()**
카운트 인코딩: 각 범주의 등장 빈도를 숫자로 변환
- 예: 'male'이 100,000번 나타나면 값 100,000


In [6]:
# --- Main Logic Loop ---

# 결과 파일들을 저장할 리스트
prediction_files = []

# 피처 엔지니어링에 사용할 Aggregation 정의
aggs_to_create = {
    'history_a_1': ['mean', 'std'], 'history_a_2': ['mean', 'std'], 
    'history_a_3': ['mean', 'std'], 'history_a_6': ['mean', 'std'],
    'feat_e_2': ['mean', 'std'], 'feat_e_3': ['mean', 'std'],
    'feat_c_8' : ['mean', 'std'], 'feat_e_9' : ['mean', 'std'],
    'feat_d_4' : ['mean', 'std'], 
    'l_feat_1' : ['mean', 'std'], 'l_feat_2' : ['mean', 'std'],
    'l_feat_5' : ['mean', 'std'], 'l_feat_7' : ['mean', 'std'],
    'l_feat_10' : ['mean', 'std'], 'l_feat_15' : ['mean', 'std']
}

for seed in CFG.SEEDS:
    print(f"\n{'='*40}\n🚀 Processing Seed: {seed}\n{'='*40}")
    seed_everything(seed)
    
    # 1. 데이터 로드 (매 반복마다 원본 로드)
    # 10% 샘플링된 데이터 사용 (전체 데이터 사용 시 파일명을 'train.parquet'으로 변경)
    train_path = os.path.join(CFG.DATA_PATH, 'train_sample_10pct.parquet') 
    print(f"📂 Loading train data from: {train_path}")
    train_df = pd.read_parquet(train_path)
    print(f"✅ Train data loaded: {train_df.shape}")
    
    # 테스트 데이터 로드
    try:
        test_df = pd.read_parquet(os.path.join(CFG.DATA_PATH, 'test.parquet'))
        print(f"✅ Test data loaded: {test_df.shape}")
    except FileNotFoundError:
        print("⚠️ Test data not found. Creating dummy test data for demo.")
        test_df = train_df.head(100).drop(columns=['clicked'])
        test_df['ID'] = [f'TEST_{i}' for i in range(len(test_df))]
    
    # 2. Down-sampling (1:1 비율)
    print("🔻 Down-sampling to 1:1 ratio...")
    clicked_1 = train_df[train_df['clicked'] == 1]
    clicked_0 = train_df[train_df['clicked'] == 0].sample(n=len(clicked_1), random_state=seed)
    train_df = pd.concat([clicked_1, clicked_0], axis=0).sample(frac=1, random_state=seed).reset_index(drop=True)
    print(f"✅ Down-sampled train data: {train_df.shape}")
    
    # 3. Feature Engineering
    print("🛠️ Adding sequence features...")
    train_df = add_seq_features(train_df, "train")
    test_df = add_seq_features(test_df, "test")
    
    # 4. Train/Validation Split
    X_train, X_val, y_train, y_val = train_test_split(
        train_df, train_df['clicked'], test_size=CFG.VALID_RATIO, random_state=seed, stratify=train_df['clicked']
    )
    
    print(f"✅ Train/Val split: {X_train.shape} / {X_val.shape}")
    
    # 5. 특성 준비
    cat_cols = ['gender', 'age_group', 'inventory_id', 'day_of_week', 'hour']
    feature_cols = [c for c in X_train.columns if c not in ['clicked', 'ID', 'seq']]
    
    # 결측치 처리 및 타입 변환
    for col in feature_cols:
        if col in cat_cols:
            # Categorical: NaN을 'unknown'으로 변환 및 문자열화
            X_train[col] = X_train[col].fillna('unknown').astype(str)
            X_val[col] = X_val[col].fillna('unknown').astype(str)
            test_df[col] = test_df[col].fillna('unknown').astype(str)
        else:
            # Numeric: NaN을 0으로 채우기 및 float로 변환
            X_train[col] = pd.to_numeric(X_train[col], errors='coerce').fillna(0).astype(float)
            X_val[col] = pd.to_numeric(X_val[col], errors='coerce').fillna(0).astype(float)
            test_df[col] = pd.to_numeric(test_df[col], errors='coerce').fillna(0).astype(float)
    
    print(f"🔥 Training CatBoost with {len(feature_cols)} features (cat_features: {cat_cols})...")
    
    model = cb.CatBoostClassifier(
        iterations=100,
        learning_rate=0.05,
        depth=6,
        loss_function='Logloss',
        cat_features=cat_cols,
        verbose=50,
        random_state=seed,
        task_type='CPU'
    )
    
    model.fit(
        X_train[feature_cols], y_train,
        eval_set=(X_val[feature_cols], y_val),
        early_stopping_rounds=10,
        verbose=False
    )
    
    # 6. 예측 및 저장
    print("📊 Making predictions...")
    y_pred_val = model.predict_proba(X_val[feature_cols])[:, 1]
    y_pred_test = model.predict_proba(test_df[feature_cols])[:, 1]
    
    # Validation 성능 평가
    val_ap = average_precision_score(y_val, y_pred_val)
    val_logloss = log_loss(y_val, y_pred_val)
    print(f"✅ Validation AP: {val_ap:.4f}, LogLoss: {val_logloss:.4f}")
    
    # 예측 결과 저장
    output_file = f"{CFG.OUTPUT_PREFIX}_{seed}_predictions.csv"
    result_df = pd.DataFrame({'ID': test_df['ID'].values, 'clicked': y_pred_test})
    result_df.to_csv(output_file, index=False)
    print(f"💾 Predictions saved: {output_file}")
    
    prediction_files.append(output_file)
    gc.collect()

print(f"\n✅ All seeds processed: {prediction_files}")


🚀 Processing Seed: 42
📂 Loading train data from: /home/konkukstat/python3/workspace/data/train_sample_10pct.parquet
✅ Train data loaded: (1071335, 119)
✅ Test data loaded: (1527298, 119)
🔻 Down-sampling to 1:1 ratio...
✅ Down-sampled train data: (40546, 119)
🛠️ Adding sequence features...


Processing test: 100%|██████████| 1527298/1527298 [02:16<00:00, 11215.93it/s]


✅ Train/Val split: (32436, 123) / (8110, 123)
🔥 Training CatBoost with 121 features (cat_features: ['gender', 'age_group', 'inventory_id', 'day_of_week', 'hour'])...
📊 Making predictions...
✅ Validation AP: 0.7215, LogLoss: 0.6159
💾 Predictions saved: 03_CatBoost_42_predictions.csv

🚀 Processing Seed: 106
📂 Loading train data from: /home/konkukstat/python3/workspace/data/train_sample_10pct.parquet
✅ Train data loaded: (1071335, 119)
✅ Test data loaded: (1527298, 119)
🔻 Down-sampling to 1:1 ratio...
✅ Down-sampled train data: (40546, 119)
🛠️ Adding sequence features...


Processing test: 100%|██████████| 1527298/1527298 [02:16<00:00, 11207.65it/s]


✅ Train/Val split: (32436, 123) / (8110, 123)
🔥 Training CatBoost with 121 features (cat_features: ['gender', 'age_group', 'inventory_id', 'day_of_week', 'hour'])...
📊 Making predictions...
✅ Validation AP: 0.7233, LogLoss: 0.6131
💾 Predictions saved: 03_CatBoost_106_predictions.csv

🚀 Processing Seed: 1031
📂 Loading train data from: /home/konkukstat/python3/workspace/data/train_sample_10pct.parquet
✅ Train data loaded: (1071335, 119)
✅ Test data loaded: (1527298, 119)
🔻 Down-sampling to 1:1 ratio...
✅ Down-sampled train data: (40546, 119)
🛠️ Adding sequence features...


Processing test: 100%|██████████| 1527298/1527298 [02:13<00:00, 11408.93it/s]


✅ Train/Val split: (32436, 123) / (8110, 123)
🔥 Training CatBoost with 121 features (cat_features: ['gender', 'age_group', 'inventory_id', 'day_of_week', 'hour'])...
📊 Making predictions...
✅ Validation AP: 0.7054, LogLoss: 0.6240
💾 Predictions saved: 03_CatBoost_1031_predictions.csv

✅ All seeds processed: ['03_CatBoost_42_predictions.csv', '03_CatBoost_106_predictions.csv', '03_CatBoost_1031_predictions.csv']


## TossMetric: 커스텀 평가 지표

Toss 대회에서 정의한 목표 함수:
$$\text{Score} = 0.5 \times AP + 0.5 \times \frac{1}{1 + \text{WeightedLogLoss}}$$

- **AP (Average Precision)**: 클릭한 상품을 정확히 예측하는 능력 측정
- **Weighted LogLoss**: 클래스 불균형을 고려한 손실 함수
  - 클릭(1)과 비클릭(0)에 동일한 가중치 부여
  - 클릭이 매우 드물어서 단순 LogLoss만으로는 모델이 모든 값을 0으로 예측하려는 경향


In [7]:

# --- Soft Voting Ensemble ---
print(f"\n🤝 Performing Soft Voting on {len(prediction_files)} files: {prediction_files}")

if len(prediction_files) > 0:
    merged = pd.read_csv(prediction_files[0]).rename(columns={"clicked": f"click_{CFG.SEEDS[0]}"})
    
    for i, file in enumerate(prediction_files[1:], start=1):
        df_temp = pd.read_csv(file).rename(columns={"clicked": f"click_{CFG.SEEDS[i]}"})
        merged = merged.merge(df_temp, on="ID")
    
    # Mean
    click_cols = [c for c in merged.columns if c.startswith("click_")]
    merged["clicked"] = merged[click_cols].mean(axis=1)
    
    # Save Final
    final_output = "03_CatBoost_SoftVoting.csv"
    merged[["ID", "clicked"]].to_csv(final_output, index=False)
    print(f"🎉 Final CatBoost Ensemble Saved: {final_output}")
    print(merged.head())
else:
    print("❌ No prediction files found.")



🤝 Performing Soft Voting on 3 files: ['03_CatBoost_42_predictions.csv', '03_CatBoost_106_predictions.csv', '03_CatBoost_1031_predictions.csv']
🎉 Final CatBoost Ensemble Saved: 03_CatBoost_SoftVoting.csv
             ID  click_42  click_106  click_1031   clicked
0  TEST_0000000  0.424046   0.414745    0.468882  0.435891
1  TEST_0000001  0.381192   0.371616    0.390215  0.381008
2  TEST_0000002  0.453006   0.465260    0.443599  0.453955
3  TEST_0000003  0.452333   0.466632    0.461494  0.460153
4  TEST_0000004  0.411486   0.420445    0.417225  0.416385


## Soft Voting 결과 해석

### 📊 결과 테이블 분석

| ID | Seed 42 | Seed 106 | Seed 1031 | **최종 예측** |
|----|---------|----------|-----------|------------|
| TEST_0000000 | 0.4240 | 0.4147 | 0.4689 | **0.4359** |
| TEST_0000001 | 0.3812 | 0.3716 | 0.3902 | **0.3810** |
| TEST_0000002 | 0.4530 | 0.4653 | 0.4436 | **0.4540** |
| TEST_0000003 | 0.4523 | 0.4666 | 0.4615 | **0.4602** |
| TEST_0000004 | 0.4115 | 0.4204 | 0.4172 | **0.4164** |

### 🔍 각 열의 의미

- **click_42, click_106, click_1031**: 각각 다른 난수 시드로 초기화된 3개 CatBoost 모델의 예측 확률
- **clicked**: 3개 모델 예측값의 평균 (최종 앙상블 결과)

$$\text{최종 예측} = \frac{\text{click\_42} + \text{click\_106} + \text{click\_1031}}{3}$$

### 💡 해석 포인트

#### 1️⃣ **모델 간 예측 편차가 작을수록 신뢰도 높음**
```
TEST_0000001:  편차 = 0.3716 ~ 0.3902 = 0.0186 (작음 ✅)
TEST_0000000:  편차 = 0.4147 ~ 0.4689 = 0.0542 (큼 ⚠️)
```
- TEST_0000001: 3개 모델이 거의 같은 예측 → 높은 신뢰도
- TEST_0000000: 3개 모델이 다른 예측 → 낮은 신뢰도 (조사 필요)

#### 2️⃣ **예측값 범위**
- 0.38 ~ 0.46: 중간 정도의 클릭 확률
- 0.5 이상이면 클릭할 가능성 높음
- 0.5 미만이면 클릭할 가능성 낮음

#### 3️⃣ **앙상블의 역할**
- **개별 모델**: 과적합 위험, 불안정성
- **Soft Voting**: 3개 모델의 장점 결합, 예측 안정화
- **결과**: 더 신뢰할 수 있는 최종 예측

### ✅ 저장된 파일

`03_CatBoost_SoftVoting.csv`: ID와 최종 예측 확률만 포함
- 형식: (ID, clicked) 2개 컬럼
- 다음 단계: 다른 모델(XGBoost, FiBiNet)과 최종 가중 앙상블 예정


## Soft Voting 앙상블 (3개 모델 결합)

**목표**: 3개 모델의 예측 확률을 가중 평균하여 최종 예측값 생성

### 프로세스

1. **예측 파일 로드**
   - `03_CatBoost_42_predictions.csv`
   - `03_CatBoost_106_predictions.csv`
   - `03_CatBoost_1031_predictions.csv`

2. **ID 기준 병합**
   - 모든 모델의 예측값을 동일한 ID 행으로 정렬

3. **가중 평균 계산**
   - 3개 모델이 각각 어떤 확률을 예측했는지 평균 계산
   - 예: (0.42 + 0.45 + 0.38) / 3 = 0.42

4. **최종 파일 저장**
   - `03_CatBoost_SoftVoting.csv`
   - 형식: ID, 앙상블 예측 확률

### 앙상블의 이점
- **안정성**: 3개 모델의 예측이 다르면 과적합 위험 감소
- **일반화**: 여러 가중 초기화를 통한 다양성 확보
- **성능**: 보통 단일 모델보다 더 좋은 성능 달성


## 메인 학습 루프 (3개 시드 X 3회 반복)

**목표**: 서로 다른 초기값으로 3개의 독립적인 CatBoost 모델 학습 → 앙상블

### 파이프라인 단계

#### 1️⃣ **데이터 로드**
- 학습 데이터: `train_sample_10pct.parquet` (약 107만 행, 119개 컬럼)
- 테스트 데이터: `test.parquet` (약 153만 행)

#### 2️⃣ **Down-Sampling (클래스 불균형 해결)**
- 원본: 비클릭(98%) vs 클릭(2%) → **1:50 불균형**
- 처리: 클릭 샘플 수만큼 비클릭 샘플 랜덤 추출 → **1:1 균형**
- 결과: 약 40K 행으로 축소 (학습 속도 향상)

#### 3️⃣ **Feature Engineering**
- `add_seq_features()`: 사용자 시퀀스 → 4개 파생 변수
- 결측치 처리 및 타입 변환

#### 4️⃣ **Train/Validation 분할**
- 8:2 비율로 분할 (80% 학습, 20% 검증)
- Stratified 분할: 클릭/비클릭 비율 유지

#### 5️⃣ **특성 준비**
- Categorical: `['gender', 'age_group', 'inventory_id', 'day_of_week', 'hour']`
- Numeric: 나머지 모든 컬럼
- NaN 처리: 범주형은 'unknown', 수치형은 0으로 채우기

#### 6️⃣ **CatBoost 모델 학습**
- **iterations**: 100 (부스팅 반복 횟수)
- **learning_rate**: 0.05 (각 부스팅 스텝의 영향력)
- **depth**: 6 (트리 깊이 = 피처 상호작용 복잡도)
- **early_stopping_rounds**: 10 (검증 손실이 10번 연속 개선 없으면 멈춤)

#### 7️⃣ **예측 및 평가**
- 검증 데이터로 `AP`와 `LogLoss` 계산
- 테스트 데이터 예측값 저장 → `03_CatBoost_{seed}_predictions.csv`

#### 8️⃣ **메모리 관리**
- 각 반복 후 `gc.collect()`: 메모리 누수 방지
